Initialize parser

In [5]:
from tree_sitter import Language, Parser
from pathlib import Path

LIB = "../../build/tree-sitter-binaries/kotlin.so"
KOTLIN_LANGUAGE = Language(LIB, "kotlin")

parser = Parser()
parser.set_language(KOTLIN_LANGUAGE)

Parsing Kt #1 - Preview

In [6]:
kt_1 = Path("../../resources/golden-reference/original/test1-preview.kt").read_bytes()
tree = parser.parse(kt_1)
root_node = tree.root_node

is_valid_kotlin = not root_node.has_error

query = KOTLIN_LANGUAGE.query("""
    ; Pattern 1: Standard declarations
    (function_declaration (simple_identifier) @func_name)

    ; Pattern 2: Expression-style (infix 'fun')
    (infix_expression
        (simple_identifier) @keyword_fun
        (#eq? @keyword_fun "fun")
        (call_expression (simple_identifier) @func_name)
    )
""")

all_functions = []

for node, capture_name in query.captures(root_node):
    if capture_name == "func_name":
        func_name = node.text.decode('utf8')
        annotations = []

        # Find the 'anchor' node to start looking for annotations
        # It's either the function_declaration or the infix_expression
        anchor = node
        while anchor and anchor.type not in ('function_declaration', 'infix_expression'):
            anchor = anchor.parent

        if anchor:
            # 1. Check for annotations inside modifiers (Standard Case)
            for child in anchor.children:
                if child.type == 'modifiers':
                    for mod in child.children:
                        if mod.type == 'annotation':
                            annotations.append(mod.text.decode('utf8'))

            # 2. Check for annotations in parent prefix_expressions (Expression Case)
            # This 'walks up' the tree to find all stacked annotations
            curr = anchor.parent
            while curr and curr.type in ('prefix_expression', 'annotated_lambda'):
                for child in curr.children:
                    if child.type == 'annotation':
                        ann_text = child.text.decode('utf8')
                        if ann_text not in annotations:
                            annotations.append(ann_text)
                curr = curr.parent

            all_functions.append({
                'name': func_name,
                'annotations': annotations
            })

for func_data in all_functions:
    print(f"Function: {func_data['name']}")
    if func_data['annotations']:
        print(f"  Annotations (on function):")
        # Reverse to show them in top-to-bottom order
        for ann in reversed(func_data['annotations']):
            print(f"    {ann}")
    else:
        print(f"  No annotations found")
    print()

total_preview_composable_functions = sum(
    1 for func in all_functions
    if any('@Preview' in ann for ann in func['annotations']) and
       any('@Composable' in ann for ann in func['annotations'])
)

print(f"Valid Kotlin: {is_valid_kotlin}")
print(f"Total functions found: {len(all_functions)}")
print(f"Total functions with @Preview and @Composable: {total_preview_composable_functions}")

Function: InterestsCardPreview
  Annotations (on function):
    @Preview
    @Composable

Function: InterestsCardLongNamePreview
  Annotations (on function):
    @Preview
    @Composable

Function: InterestsCardLongDescriptionPreview
  Annotations (on function):
    @Preview
    @Composable

Function: InterestsCardWithEmptyDescriptionPreview
  Annotations (on function):
    @Preview
    @Composable

Function: InterestsCardSelectedPreview
  Annotations (on function):
    @Composable
    @Preview

Valid Kotlin: True
Total functions found: 5
Total functions with @Preview and @Composable: 5


Parsing Kt #2 - Unit Test

In [7]:
kt_2 = Path("../../resources/golden-reference/original/test2-unit-test.kt").read_bytes()
tree = parser.parse(kt_2)

root_node = tree.root_node

is_valid_kotlin = not root_node.has_error

query = KOTLIN_LANGUAGE.query("""
    ; 1. Capture all import paths
    (import_header (identifier) @import_path)

    ; 2. Capture function names and their surrounding annotations
    (function_declaration (simple_identifier) @func_name)
    (infix_expression
        (simple_identifier) @kw_fun (#eq? @kw_fun "fun")
        (call_expression (simple_identifier) @func_name))

    ; 3. Capture constructor calls in properties (e.g., val x = ClassName())
    (property_declaration
        (variable_declaration (simple_identifier) @prop_name)
        (call_expression (simple_identifier) @constructor_name))
""")

results = {
    "imports": [],
    "functions": [],
    "constructors": []
}

for node, capture_name in query.captures(root_node):
    text = node.text.decode('utf8')

    if capture_name == "import_path":
        results["imports"].append(text)

    elif capture_name == "func_name":
        annotations = []
        anchor = node
        # Find the function declaration or infix base
        while anchor and anchor.type not in ('function_declaration', 'infix_expression'):
            anchor = anchor.parent

        if anchor:
            # 1. Internal Check (Inside the function modifiers) ---
            # Most common for standard classes
            for child in anchor.children:
                if child.type == 'modifiers':
                    for mod in child.children:
                        if mod.type == 'annotation':
                            annotations.append(mod.text.decode('utf8'))

            # 2. External Check (Climbing prefix_expressions) ---
            # Common for Compose or expression-style definitions
            curr = anchor.parent
            while curr and curr.type in ('prefix_expression', 'annotated_lambda'):
                for child in curr.children:
                    if child.type == 'annotation':
                        ann_text = child.text.decode('utf8')
                        if ann_text not in annotations:
                            annotations.append(ann_text)
                curr = curr.parent

        results["functions"].append({"name": text, "annotations": annotations})

    elif capture_name == "constructor_name":
        results["constructors"].append(text)

required_imports = [
    "com.google.samples.apps.nowinandroid.core.model.data.FollowableTopic",
    "com.google.samples.apps.nowinandroid.core.model.data.Topic",
    "com.google.samples.apps.nowinandroid.core.testing.repository.TestTopicsRepository",
    "com.google.samples.apps.nowinandroid.core.testing.repository.TestUserDataRepository"
]
has_all_required_imports = all(req_import in results['imports'] for req_import in required_imports)

has_use_case_constructor = "GetFollowableTopicsUseCase" in results['constructors']

test_functions_count = sum(
    1 for f in results['functions']
    if any("@Test" in a for a in f['annotations'])
)
has_exactly_three_test_functions = (test_functions_count == 2)

print(f"--- Analysis ---")
print(f"Valid Kotlin: {is_valid_kotlin}")
print(f"Has All Required Imports: {has_all_required_imports}")
print(f"Has GetFollowableTopicsUseCase Constructor: {has_use_case_constructor}")
print(f"Functions with @Test annotation: {test_functions_count}")
print(f"Has Exactly 2 Test Functions: {has_exactly_three_test_functions}")

--- Analysis ---
Valid Kotlin: True
Has All Required Imports: True
Has GetFollowableTopicsUseCase Constructor: True
Functions with @Test annotation: 2
Has Exactly 2 Test Functions: True


Parsing Kt #3 - Instrumentation Test

In [8]:
kt_3 = Path("../../resources/golden-reference/original/test3-instrumentation-test.kt").read_bytes()
tree = parser.parse(kt_3)
root_node = tree.root_node

is_valid_kotlin = not root_node.has_error

query = KOTLIN_LANGUAGE.query("""
    (import_header (identifier) @import_path)

    (function_declaration (simple_identifier) @func_name)
    (infix_expression
        (simple_identifier) @kw_fun (#eq? @kw_fun "fun")
        (call_expression (simple_identifier) @func_name))

    (class_declaration
        (type_identifier) @class_name
        (delegation_specifier
            (constructor_invocation
                (user_type
                    (type_identifier) @parent_class)))?
        (class_body)?)
""")

results = {
    "imports": [],
    "functions": [],
    "classes": {}
}

for node, capture_name in query.captures(root_node):
    text = node.text.decode('utf8')

    if capture_name == "import_path":
        results["imports"].append(text)

    elif capture_name == "func_name":
        annotations = []
        anchor = node
        while anchor and anchor.type not in ('function_declaration', 'infix_expression'):
            anchor = anchor.parent

        if anchor:
            for child in anchor.children:
                if child.type == 'modifiers':
                    for mod in child.children:
                        if mod.type == 'annotation':
                            annotations.append(mod.text.decode('utf8'))
            curr = anchor.parent
            while curr and curr.type in ('prefix_expression', 'annotated_lambda'):
                for child in curr.children:
                    if child.type == 'annotation':
                        if (ann := child.text.decode('utf8')) not in annotations:
                            annotations.append(ann)
                curr = curr.parent

        results["functions"].append({"name": text, "annotations": annotations})

    elif capture_name == "class_name":
        if text not in results["classes"]:
            results["classes"][text] = {"parents": []}

    elif capture_name == "parent_class":
        # Find which class this parent belongs to by traversing up
        curr = node
        while curr:
            if curr.type == 'class_declaration':
                for child in curr.children:
                    if child.type == 'type_identifier':
                        class_name = child.text.decode('utf8')
                        if class_name in results["classes"]:
                            results["classes"][class_name]["parents"].append(text)
                        break
                break
            curr = curr.parent

results["classes"] = [{"name": k, "parents": v["parents"]} for k, v in results["classes"].items()]

has_topic_entity_import = any("TopicEntity" in imp for imp in results["imports"])
uses_database_test = any("DatabaseTest" in c["parents"] for c in results["classes"])
test_functions = [f['name'] for f in results['functions'] if any("@Test" in a for a in f['annotations'])]

print(f"--- Analysis ---")
print(f"Imports TopicEntity: {has_topic_entity_import}")
print(f"Inherits from DatabaseTest: {uses_database_test}")
print(f"Functions with @Test: {len(test_functions)}")
print(f"\n--- Details ---")
print(f"Imports: {results['imports']}")
print(f"Classes: {results['classes']}")
print(f"Test functions: {test_functions}")

--- Analysis ---
Imports TopicEntity: True
Inherits from DatabaseTest: True
Functions with @Test: 7

--- Details ---
Imports: ['com.google.samples.apps.nowinandroid.core.database.model.TopicEntity', 'kotlinx.coroutines.flow.first', 'kotlinx.coroutines.test.runTest', 'org.junit.Test', 'kotlin.test.assertEquals']
Classes: [{'name': 'TopicDaoTest', 'parents': ['DatabaseTest']}]
Test functions: ['getTopics', 'getTopic', 'getTopics_oneOff', 'getTopics_byId', 'insertTopic_newEntryIsIgnoredIfAlreadyExists', 'upsertTopic_existingEntryIsUpdated', 'deleteTopics_byId_existingEntriesAreDeleted']
